## 1. **缺失值处理**
使用 Python 的 pandas 库来检测 CSV 文件中的缺失值
这个代码会读取 weatherHistory.csv 文件并输出每一列中的缺失值数量。可以根据输出结果查看数据集中是否存在缺失值以及它们分布在哪些列。

In [ ]:
import pandas as pd

# 读取 CSV 文件
df = pd.read_csv('weatherHistory.csv')

# 检查每列的缺失值
missing_values = df.isnull().sum()

# 输出每列缺失值的数量
print("缺失值检测结果：")
print(missing_values)

缺失值检测结果：
Formatted Date                0
Summary                       0
Precip Type                 517
Temperature (C)               0
Apparent Temperature (C)      0
Humidity                      0
Wind Speed (km/h)             0
Wind Bearing (degrees)        0
Visibility (km)               0
Loud Cover                    0
Pressure (millibars)          0
Daily Summary                 0
dtype: int64


可以看到Precip Type有517个数据为null，经检查都是rain，替换成rain
使用 pandas 库中的 fillna() 函数将 Precip Type 列中的缺失值替换为 'rain'

In [51]:
import pandas as pd

# 读取 CSV 文件
print("正在读取文件 weatherHistory.csv...")
df = pd.read_csv('weatherHistory.csv')
print("文件读取成功！")

# 检查 Precip Type 列中的缺失值数量
missing_before = df['Precip Type'].isnull().sum()
print(f"替换之前，Precip Type 列中有 {missing_before} 个缺失值。")

# 将 Precip Type 列中的缺失值替换为 'rain'
df['Precip Type'] = df['Precip Type'].fillna('rain')

# 检查替换结果
missing_after = df['Precip Type'].isnull().sum()
print(f"替换之后，Precip Type 列中有 {missing_after} 个缺失值。")

# 直接覆盖保存到原始 CSV 文件
print("正在保存更改到文件 weatherHistory.csv...")
df.to_csv('weatherHistory.csv', index=False)
print("保存成功！")

# 输出保存后的信息
print("文件已更新，缺失值已处理完毕。")



正在读取文件 weatherHistory.csv...
文件读取成功！
替换之前，Precip Type 列中有 517 个缺失值。
替换之后，Precip Type 列中有 0 个缺失值。
正在保存更改到文件 weatherHistory.csv...
保存成功！
文件已更新，缺失值已处理完毕。


## 2. **异常值识别和处理**

基于箱线图和统计特征,Loud Cover(云覆盖度)次列数据全部为0，推测是数据存在残缺或者问题，将此列删除。
使用 pandas 库来删除 Loud Cover 列。
使用 df.drop(columns=['Loud Cover'], inplace=True) 删除该列并在原数据上应用更改。

In [52]:
import pandas as pd

# 读取 CSV 文件
df = pd.read_csv('weatherHistory.csv')

# 检查 Loud Cover 列是否全部为0
if (df['Loud Cover'] == 0).all():
    print("Loud Cover 列中的值全部为 0，推测为异常数据，将该列删除。")
    
    # 删除 Loud Cover 列
    df.drop(columns=['Loud Cover'], inplace=True)

# 保存删除后的数据到原始文件
df.to_csv('weatherHistory.csv', index=False)

# 输出确认删除后的数据列名
print("更新后的数据列：")
print(df.columns)


Loud Cover 列中的值全部为 0，推测为异常数据，将该列删除。
更新后的数据列：
Index(['Formatted Date', 'Summary', 'Precip Type', 'Temperature (C)',
       'Apparent Temperature (C)', 'Humidity', 'Wind Speed (km/h)',
       'Wind Bearing (degrees)', 'Visibility (km)', 'Pressure (millibars)',
       'Daily Summary'],
      dtype='object')


## 3. **去重**

Formatted Date（日期时间）不可能重复。需要检查其是否存在重复。


In [53]:
import pandas as pd

# 读取 CSV 文件
df = pd.read_csv('weatherHistory.csv')

# 查找重复的 Formatted Date
duplicate_rows = df[df['Formatted Date'].duplicated(keep=False)]

# 检查是否存在重复值并打印出来
if not duplicate_rows.empty:
    print(f"发现 {len(duplicate_rows)} 行重复的日期时间记录：")
    print(duplicate_rows)
else:
    print("没有发现重复的日期时间记录。")


发现 48 行重复的日期时间记录：
                      Formatted Date        Summary Precip Type  \
8040   2010-08-02 00:00:00.000 +0200          Clear        rain   
8041   2010-08-02 01:00:00.000 +0200          Clear        rain   
8042   2010-08-02 02:00:00.000 +0200          Clear        rain   
8043   2010-08-02 03:00:00.000 +0200          Clear        rain   
8044   2010-08-02 04:00:00.000 +0200          Clear        rain   
8045   2010-08-02 05:00:00.000 +0200          Clear        rain   
8046   2010-08-02 06:00:00.000 +0200          Clear        rain   
8047   2010-08-02 07:00:00.000 +0200          Clear        rain   
8048   2010-08-02 08:00:00.000 +0200          Clear        rain   
8049   2010-08-02 09:00:00.000 +0200          Clear        rain   
8050   2010-08-02 10:00:00.000 +0200          Clear        rain   
8051   2010-08-02 11:00:00.000 +0200          Clear        rain   
8052   2010-08-02 12:00:00.000 +0200          Clear        rain   
8053   2010-08-02 13:00:00.000 +0200  Partly

Formatted Date（日期时间）确实是有重复数据，但是在保留了第一次出现的数据之后，还需要把数据按照时间顺序排序。
先去除 Formatted Date 列中的重复数据（保留首次出现的记录），然后根据 Formatted Date 进行排序。

In [71]:
import pandas as pd

# 读取 CSV 文件
df = pd.read_csv('weatherHistory.csv')

# 将 'Formatted Date' 列转换为 UTC datetime 类型
df['Formatted Date'] = pd.to_datetime(df['Formatted Date'], utc=True)

# 将时间转换为 UTC+1
df['Formatted Date'] = df['Formatted Date'] + pd.Timedelta(hours=1)

# 将时间转换回带时区的字符串格式
df['Formatted Date'] = df['Formatted Date'].dt.strftime('%Y-%m-%d %H:%M:%S') + '+01:00'

# 去重 'Formatted Date' 列
df = df.drop_duplicates(subset=['Formatted Date'])

# 保存修改后的数据到 CSV 文件
df.to_csv('weatherHistory.csv', index=False)

print("处理完成，已保存为 'weatherHistory.csv'")


处理完成，已保存为 'weatherHistory_adjusted.csv'


值得注意的是，在某些地区，冬季和夏季时间
会有所不同，例如：
一些使用+01:00的国家在夏季时可能会调整到+02:00（例如，欧洲的夏时制）。这种情况下，在夏季某些地区的时间可能会被认为相同（例如，冬季的+01:00变为+02:00)，但在标准时间上它们是不同的。
例如，欧洲中部时间（Central European Time, CET）是一个典型的例子。
我们将把时间数据统一转换为标准时区 UTC+01:00，即欧洲中部标准时间（CET），不再考虑夏令时的变动。这样做有助于确保数据分析时的时区一致性。